In [1]:
import pandas as pd
codepoint_dir = r"/Users/robinlinacre/Downloads/codepo_gb"

# Get column names
column_headers_df = pd.read_csv("/Users/robinlinacre/Downloads/codepo_gb/Doc/Code-Point_Open_Column_Headers.csv")
headers = column_headers_df.loc[0]
headers = [h.lower() for h in list(headers)]

In [2]:
# Iterate through the CSVs in codepoint open concatenating them together into one big table
import os 
files = os.listdir(os.path.join(codepoint_dir,"Data/CSV"))

dfs = []
for f in files:
    this_file = os.path.join(codepoint_dir,"Data/CSV", f)
    if ".csv" in this_file:
        this_df = pd.read_csv(this_file, header=None)
        dfs.append(this_df)

In [3]:
final_df = pd.concat(dfs)
final_df.columns = headers

In [4]:
len(final_df)

1696113

In [5]:
from mylibrary.connections import engine, cursor, conn

/Users/robinlinacre/anaconda/lib/python3.6/site-packages/sqlalchemy/dialects/postgresql/base.py:2526: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))


In [6]:
final_df.to_sql("all_postcodes", engine, schema="tt_h", if_exists="replace")

In [8]:
# Create geometry column for the points including a spatial index for efficient querying
sql = """
SELECT AddGeometryColumn ('tt_h', 'all_postcodes', 'geom', 27700, 'POINT', 2);
UPDATE tt_h.all_postcodes SET geom = ST_GeomFromText('POINT(' || eastings || ' ' || northings || ')', 27700 );
CREATE INDEX idx_geom_all_postcodes_tt_gh ON tt.all_postcodes USING gist(geom);
"""

cursor.execute(sql)
conn.commit()

InternalError: current transaction is aborted, commands ignored until end of transaction block


In [ ]:
sql = """
ALTER TABLE tt_h.all_postcodes ADD lat float, ADD lng float;

UPDATE tt_h.all_postcodes SET
    lng = ST_X(ST_TRANSFORM(geom, 4326)),
    lat = ST_Y(ST_TRANSFORM(geom,4326));

""" 
cursor.execute(sql)
conn.commit()